# Get players data

In [1]:
import pandas as pd
import time
from urllib.error import HTTPError


def process_player_stats(player_url, player_name):
    # Fetch the data from the given URL
    player_data = pd.read_html(player_url)[0]
    
    # Drop unnecessary rows and columns
    player_data.columns = player_data.columns.droplevel(level=0)
    player_data = player_data.drop(player_data.index[-1])
    
    # Convert 'Min' column to numeric
    player_data['Min'] = pd.to_numeric(player_data['Min'], errors='coerce')
    
    # Keep where Min above 60
    player_data = player_data[player_data['Min'] > 60]
    
    # Keep selected columns
    player_data = player_data[['Result', 'Gls', 'Ast', 'Sh', 'SoT']]
    
    # Convert 'Gls', 'Ast', 'Sh', and 'SoT' columns to numeric
    numeric_columns = ['Gls', 'Ast', 'Sh', 'SoT']
    player_data[numeric_columns] = player_data[numeric_columns].apply(pd.to_numeric, errors='coerce')
    
    # Keep first value of result
    player_data['Result'] = player_data['Result'].str[0]
    
    # Convert to numeric, D = 0, W = 3, L = 0
    player_data['Result'] = player_data['Result'].map({'W': 3, 'D': 1, 'L': 0})
    
    # Sum of each column
    player_summary = player_data.sum(numeric_only=True)
    
    # Create a DataFrame
    player_summary_df = pd.DataFrame(player_summary).T
    
    # Add a column with the player's name
    player_summary_df['Player'] = player_name
    
    # Rename the Result column to Team_points
    player_summary_df = player_summary_df.rename(columns={'Result': 'Team_points'})
    
    # Reorder columns
    player_summary_df = player_summary_df[['Player', 'Team_points', 'Gls', 'Ast', 'Sh', 'SoT']]
    
    return player_summary_df

# List of players and their URLs
player_list = [
    ('Aleksandar Mitrović', 'https://fbref.com/en/players/3925dbd6/matchlogs/2021-2022/c10/Aleksandar-Mitrovic-Match-Logs'),
    ('André Watshini Bukia','https://fbref.com/en/players/f9a92ea5/matchlogs/2021-2022/c32/Andre-Bukia-Match-Logs'),
    ('Bas Dost', 'https://fbref.com/en/players/dc90cfbe/matchlogs/2021-2022/c37/Bas-Dost-Match-Logs'),
    ('Benik Afobe', 'https://fbref.com/en/players/b63f4077/matchlogs/2021-2022/c10/Benik-Afobe-Match-Logs'),
    ('Benito Raman', 'https://fbref.com/en/players/867501bf/matchlogs/2021-2022/Benito-Raman-Match-Logs'),
    ('Carlton Morris','https://fbref.com/en/players/2da834f1/matchlogs/2021-2022/c10/Carlton-Morris-Match-Logs'),
    ('Cauley Woodrow','https://fbref.com/en/players/d3085b49/matchlogs/2021-2022/c10/Cauley-Woodrow-Match-Logs'),
    ('Chris Martin','https://fbref.com/en/players/c21e39f9/matchlogs/2021-2022/c10/Chris-Martin-Match-Logs'),
    ('Dominic Solanke','https://fbref.com/en/players/e77dc3b2/matchlogs/2021-2022/c10/Dominic-Solanke-Match-Logs'),
    ('Duncan Watmore','https://fbref.com/en/players/2b2bae51/matchlogs/2021-2022/c10/Duncan-Watmore-Match-Logs'),
    ('Jakob Jantscher','https://fbref.com/en/players/2c5f04ba/matchlogs/2021-2022/c56/Jakob-Jantscher-Match-Logs'),
    ('James Collins','https://fbref.com/en/players/95471a13/matchlogs/2021-2022/c10/James-Collins-Match-Logs'),
    ('Jean-Luc Mamadou Diarra Dompé','https://fbref.com/en/players/9ffdf29a/matchlogs/2021-2022/Jean-Luc-Dompe-Match-Logs'),
    ('Jelle Vossen','https://fbref.com/en/players/07ab6542/matchlogs/2021-2022/Jelle-Vossen-Match-Logs'),
    ('Jhon Eduard Murillo Romaña','https://fbref.com/en/players/0d93febc/matchlogs/2021-2022/c32/Jhon-Murillo-Match-Logs'),
    ('Josh Magennis','https://fbref.com/en/players/0b40ae38/matchlogs/2021-2022/c10/Josh-Magennis-Match-Logs'),
    ('Kenji Gorré','https://fbref.com/en/players/52d1a4c6/matchlogs/2021-2022/c32/Kenji-Gorre-Match-Logs'),
    ('Laurent Depoitre','https://fbref.com/en/players/aa5859cb/matchlogs/2021-2022/c37/Laurent-Depoitre-Match-Logs'),
    ('Lawrence Shankland','https://fbref.com/en/players/63b1a176/matchlogs/2021-2022/c37/Lawrence-Shankland-Match-Logs'),
    ('Lewis Grabban','https://fbref.com/en/players/1a857282/matchlogs/2021-2022/c10/Lewis-Grabban-Match-Logs'),
    ('Lior Refaelov','https://fbref.com/en/players/f754c59e/matchlogs/2021-2022/c37/Lior-Refaelov-Match-Logs'),
    ('Lucas Eduardo Santos João','https://fbref.com/en/players/9ddd4e40/matchlogs/2021-2022/Lucas-Joao-Match-Logs'),
    ('Lukas Jutkiewicz','https://fbref.com/en/players/3943a06f/matchlogs/2021-2022/Lukas-Jutkiewicz-Match-Logs'),
    ('Manuel Hedilazio Benson','https://fbref.com/en/players/d7f99582/matchlogs/2021-2022/c37/Benson-Manuel-Match-Logs'),
    ('Michael Frey','https://fbref.com/en/players/33182cb8/matchlogs/2021-2022/c37/Michael-Frey-Match-Logs'),
    ('Neeskens Kebano', 'https://fbref.com/en/players/18b999f4/matchlogs/2021-2022/c10/Neeskens-Kebano-Match-Logs'),
    ('Pablo Sarabia García','https://fbref.com/en/players/9744ff80/matchlogs/2021-2022/c32/Pablo-Sarabia-Match-Logs'),
    ('João Paulo Dias Fernandes','https://fbref.com/en/players/a62c16d0/matchlogs/2021-2022/c32/Paulinho-Match-Logs'),
    ('Rafael Alexandre Fernandes Ferreira da Silva','https://fbref.com/en/players/70cf63ca/matchlogs/2021-2022/c32/Rafa-Silva-Match-Logs'),
    ('Clecildo Rafael Martins de Souza Ladislau','https://fbref.com/en/players/b7e1c3b5/matchlogs/2021-2022/Rafael-Martins-Match-Logs'),
    ('Ravel  Morrison','https://fbref.com/en/players/bf59c794/matchlogs/2021-2022/c10/Ravel-Morrison-Match-Logs'),
    ('Renaud Emond','https://fbref.com/en/players/5c9ac7f0/matchlogs/2021-2022/c37/Renaud-Emond-Match-Logs'),
    ('Ricardo Jorge Luz Horta','https://fbref.com/en/players/6603fed7/matchlogs/2021-2022/c32/Ricardo-Horta-Match-Logs'),
    ('Rui Pedro da Rocha Fonte','https://fbref.com/en/players/6ff8ef56/matchlogs/2021-2022/Rui-Fonte-Match-Logs'),
    ('Salvador José Milhazes Agra','https://fbref.com/en/players/a130741a/matchlogs/2021-2022/Salvador-Agra-Match-Logs'),
    ('Sam Gallagher','https://fbref.com/en/players/44361ade/matchlogs/2021-2022/c10/Sam-Gallagher-Match-Logs'),
    ('Steven Fletcher','https://fbref.com/en/players/2c9c3502/matchlogs/2021-2022/c10/Steven-Fletcher-Match-Logs'),
    ("Théo Bongonda Mbul'Ofeko Batombo",'https://fbref.com/en/players/0da6a13e/matchlogs/2021-2022/c37/Theo-Bongonda-Match-Logs'),
    ('Tom Ince','https://fbref.com/en/players/67d23ab0/matchlogs/2021-2022/c10/Tom-Ince-Match-Logs'),
    ('Tom Lawrence','https://fbref.com/en/players/7e2f45f4/matchlogs/2021-2022/c10/Tom-Lawrence-Match-Logs'),
    ('Troy Deeney','https://fbref.com/en/players/82e34b69/matchlogs/2021-2022/c10/Troy-Deeney-Match-Logs'),
    ('Xavier Mercier','https://fbref.com/en/players/de89f3c3/matchlogs/2021-2022/Xavier-Mercier-Match-Logs'),
    ('Zinho Gano','https://fbref.com/en/players/4c537d70/matchlogs/2021-2022/c37/Zinho-Gano-Match-Logs'),
    ('Christoph Monschein','https://fbref.com/en/players/ea07ceb3/matchlogs/2021-2022/c56/Christoph-Monschein-Match-Logs'),
    ('Harry Wilson', 'https://fbref.com/en/players/c6dc9ecd/matchlogs/2021-2022/c10/Harry-Wilson-Match-Logs'),
    ('Mehdi Taremi','https://fbref.com/en/players/4c2a5d34/matchlogs/2021-2022/c32/Mehdi-Taremi-Match-Logs'),
    ('Aaron Leya Iseka','https://fbref.com/en/players/29ef0f49/matchlogs/2021-2022/c10/Aaron-Leya-Iseka-Match-Logs'),
    ('Morgan Gibbs-White','https://fbref.com/en/players/32f60ed8/matchlogs/2021-2022/c10/Morgan-Gibbs-White-Match-Logs'),
    ('Bobby De Cordova-Reid','https://fbref.com/en/players/0f7533cd/matchlogs/2021-2022/c10/Bobby-Reid-Match-Logs'),
    ('Danny Ward','https://fbref.com/en/players/cfb29823/matchlogs/2021-2022/c10/Danny-Ward-Match-Logs'),
    ('Gary Madine','https://fbref.com/en/players/e3e6502a/matchlogs/2021-2022/c10/Gary-Madine-Match-Logs'),
    ('Makhtar Gueye','https://fbref.com/en/players/92b1763b/matchlogs/2021-2022/Makhtar-Gueye-Match-Logs'),
    ('Smail Prevljak','https://fbref.com/en/players/d6f5490e/matchlogs/2021-2022/c37/Smail-Prevljak-Match-Logs'),
    ('Roman Yaremchuk','https://fbref.com/en/players/cb641b9d/matchlogs/2021-2022/c32/Roman-Yaremchuk-Match-Logs'),
    ('Rafik Guitane','https://fbref.com/en/players/7091324a/matchlogs/2021-2022/Rafik-Guitane-Match-Logs'),
    ('Antoine Leautey','https://fbref.com/en/players/f2350250/matchlogs/2021-2022/Antoine-Leautey-Match-Logs'),
    ('Callum Robinson','https://fbref.com/en/players/e75750a3/matchlogs/2021-2022/c10/Callum-Robinson-Match-Logs'),
    ('Andraž Šporar','https://fbref.com/en/players/878f0921/matchlogs/2021-2022/c10/Andraz-Sporar-Match-Logs'),
    ('Sean Maguire','https://fbref.com/en/players/7f07729c/matchlogs/2021-2022/c10/Sean-Maguire-Match-Logs'),
    ('Scott Hogan','https://fbref.com/en/players/cdda4027/matchlogs/2021-2022/c10/Scott-Hogan-Match-Logs'),
    ('Denis Drăguş','https://fbref.com/en/players/0e490f6e/matchlogs/2021-2022/Denis-Dragus-Match-Logs'),
    ('Ryan Christie','https://fbref.com/en/players/26ce2263/matchlogs/2021-2022/c10/Ryan-Christie-Match-Logs'),
    ('Thierry Ambrose','https://fbref.com/en/players/2945f4b7/matchlogs/2021-2022/Thierry-Ambrose-Match-Logs'),
    ('Otávio Edmilson da Silva Monteiro','https://fbref.com/en/players/28b1f3ce/matchlogs/2021-2022/c32/Otavio-Match-Logs'),
    ('Noa Lang','https://fbref.com/en/players/8e034340/matchlogs/2021-2022/c37/Noa-Lang-Match-Logs'),
    ('Ebere Paul Onuachu','https://fbref.com/en/players/c06d1086/matchlogs/2021-2022/c37/Paul-Onuachu-Match-Logs'),
    ('Markus Pink','https://fbref.com/en/players/b730943b/matchlogs/2021-2022/Markus-Pink-Match-Logs'),
    ('João Klauss de Mello','https://fbref.com/en/players/f5ae006d/matchlogs/2021-2022/Klauss-Match-Logs'),
    ('Noah Okafor','https://fbref.com/en/players/ca607d59/matchlogs/2021-2022/c56/Noah-Okafor-Match-Logs'),
    ('Pedro António Pereira Gonçalves','https://fbref.com/en/players/c275947b/matchlogs/2021-2022/c32/Pedro-Goncalves-Match-Logs'),
    ('Viktor Gyökeres','https://fbref.com/en/players/4d5a9185/matchlogs/2021-2022/c10/Viktor-Gyokeres-Match-Logs'),
    ('Aaron Connolly','https://fbref.com/en/players/27c01749/matchlogs/2021-2022/c10/Aaron-Connolly-Match-Logs'),
    ('Christian  Kouamé','https://fbref.com/en/players/58e933d2/matchlogs/2021-2022/c37/Christian-Kouame-Match-Logs'),
    ('André Filipe Cunha Vidigal','https://fbref.com/en/players/3f332394/matchlogs/2021-2022/Andre-Vidigal-Match-Logs'),
    ('Francis Amuzu','https://fbref.com/en/players/4f0fbd15/matchlogs/2021-2022/c37/Francis-Amuzu-Match-Logs'),
    ('Mbwana Ally Samatta','https://fbref.com/en/players/dae8623e/matchlogs/2021-2022/c37/Mbwana-Samatta-Match-Logs'),
    ('Joseph Paintsil','https://fbref.com/en/players/88062e83/matchlogs/2021-2022/c37/Joseph-Paintsil-Match-Logs'),
    ('Cameron Jerome','https://fbref.com/en/players/680fb5ee/matchlogs/2021-2022/c10/Cameron-Jerome-Match-Logs'),
    ('Leonardo Acevedo Ruiz','https://fbref.com/en/players/5ed6bf7a/matchlogs/2021-2022/Leonardo-Ruiz-Match-Logs'),
    ('Murilo De Souza Costa','https://fbref.com/en/players/85733f33/matchlogs/2021-2022/Murilo-Costa-Match-Logs'),
    ('Diogo Filipe Costa Rocha','https://fbref.com/en/players/c738a5ec/matchlogs/2021-2022/Rochinha-Match-Logs'),
    ('Yusupha Njie','https://fbref.com/en/players/51f22984/matchlogs/2021-2022/c32/Yusupha-Njie-Match-Logs'),
    ('Hélder José Castro Ferreira','https://fbref.com/en/players/6407ce4d/matchlogs/2021-2022/c32/Helder-Ferreira-Match-Logs'),
    ('Oscar Eduardo Estupiñán Vallesilla','https://fbref.com/en/players/213ab9bb/matchlogs/2021-2022/c32/Oscar-Estupinan-Match-Logs'),
    ('Rafael Avelino Pereira Pinto Barbosa','https://fbref.com/en/players/2a34574e/matchlogs/2021-2022/Rafael-Barbosa-Match-Logs'),
    ('Diederrick Joel Tagueu Tadjo','https://fbref.com/en/players/edd65ae0/matchlogs/2021-2022/c32/Zainadine-Junior-Match-Logs'),
    ('Thomas Goiginger','https://fbref.com/en/players/e1ba3d5a/matchlogs/2020-2021/c56/Thomas-Goiginger-Match-Logs'),
    ('Shamar Nicholson','https://fbref.com/en/players/e33fb79d/matchlogs/2021-2022/c37/Shamar-Nicholson-Match-Logs'),
    ('Dino Hotič','https://fbref.com/en/players/a4aa453f/matchlogs/2021-2022/c37/Dino-Hotic-Match-Logs'),
    ('Dereck Kutesa','https://fbref.com/en/players/1b6862e8/matchlogs/2021-2022/Dereck-Kutesa-Match-Logs'),
    ('Vakoun Issouf Bayo','https://fbref.com/en/players/132f161f/matchlogs/2021-2022/c37/Vakoun-Issouf-Bayo-Match-Logs'),
    ('Giacomo Vrioni','https://fbref.com/en/players/bf4d31d3/matchlogs/2021-2022/c56/Giacomo-Vrioni-Match-Logs'),
    ('Danel Sinani','https://fbref.com/en/players/96ddfa79/matchlogs/2021-2022/c10/Danel-Sinani-Match-Logs'),
    ('Atdhe Nuhiu','https://fbref.com/en/players/06f988ce/matchlogs/2021-2022/Atdhe-Nuhiu-Match-Logs'),
    ('Éverton Sousa Soares','https://fbref.com/en/players/b1cbc666/matchlogs/2021-2022/c32/Everton-Soares-Match-Logs'),
    ('Simon Bokote Banza','https://fbref.com/en/players/be47b750/matchlogs/2021-2022/c32/Simon-Banza-Match-Logs'),
    ('Billy Sharp','https://fbref.com/en/players/e8c9f4e9/matchlogs/2021-2022/c10/Billy-Sharp-Match-Logs'),
    ('Ben Brereton Díaz','https://fbref.com/en/players/57827369/matchlogs/2021-2022/c10/Ben-Brereton-Match-Logs'),
    ('Ben Osborn','https://fbref.com/en/players/79bc4c45/matchlogs/2021-2022/c10/Ben-Osborn-Match-Logs'),
    ('Nahki Wells','https://fbref.com/en/players/b070482d/matchlogs/2021-2022/c10/Nahki-Wells-Match-Logs'),
    ('Tyrese Campbell','https://fbref.com/en/players/38aefb74/matchlogs/2021-2022/c10/Tyrese-Campbell-Match-Logs'),
    ('Jed Wallace','https://fbref.com/en/players/45194ce9/matchlogs/2021-2022/c10/Jed-Wallace-Match-Logs'),
    ('Jack Marriott','https://fbref.com/en/players/e4551dab/matchlogs/2021-2022/c10/Jack-Marriott-Match-Logs'),
    ('Duane Holmes','https://fbref.com/en/players/47a050ed/matchlogs/2021-2022/c10/Duane-Holmes-Match-Logs'),
    ('Jamie Paterson','https://fbref.com/en/players/8b6bfc53/matchlogs/2021-2022/Jamie-Paterson-Match-Logs'),
    ('Jordan Hugill','https://fbref.com/en/players/a36d4785/matchlogs/2021-2022/c10/Jordan-Hugill-Match-Logs'),
    ('Keinan Davis','https://fbref.com/en/players/9deaf2c8/matchlogs/2021-2022/c10/Keinan-Davis-Match-Logs'),
    ('Marcus Edwards','https://fbref.com/en/players/57760e66/matchlogs/2021-2022/c32/Marcus-Edwards-Match-Logs'),
    ('Bertrand Yves Baraye','https://fbref.com/en/players/9cf87d33/matchlogs/2021-2022/Yves-Baraye-Match-Logs'),
    ('Thorsten  Röcher','https://fbref.com/en/players/a6d9cb9d/matchlogs/2021-2022/Thorsten-Rocher-Match-Logs'),
    ('Michael Obafemi','https://fbref.com/en/players/caa1a7f0/matchlogs/2021-2022/c10/Michael-Obafemi-Match-Logs'),
    ('Kévin Ahoueke Denkey','https://fbref.com/en/players/0d3585b5/matchlogs/2021-2022/c37/Kevin-Denkey-Match-Logs'),
    ('Karlan Ahearne-Grant','https://fbref.com/en/players/87f7500e/matchlogs/2021-2022/c10/Karlan-Grant-Match-Logs'),
    ('Sory Kaba','https://fbref.com/en/players/d29202c2/matchlogs/2021-2022/c37/Sory-Kaba-Match-Logs'),
    ('Rabbi Matondo','https://fbref.com/en/players/33144cc5/matchlogs/2021-2022/c37/Rabbi-Matondo-Match-Logs'),
    ('Yari Verschaeren','https://fbref.com/en/players/6b71ba6c/matchlogs/2021-2022/c37/Yari-Verschaeren-Match-Logs'),
    ('Youssef Maziz','https://fbref.com/en/players/514270e8/matchlogs/2021-2022/Youssef-Maziz-Match-Logs'),
    ('Thomas Sabitzer','https://fbref.com/en/players/9993817f/matchlogs/2021-2022/Thomas-Sabitzer-Match-Logs'),
    ('Renat Dadashov','https://fbref.com/en/players/06dc2511/matchlogs/2021-2022/c32/Renat-Dadasov-Match-Logs'),
    ('Shayne Lavery','https://fbref.com/en/players/d333ffb9/matchlogs/2021-2022/c10/Shayne-Lavery-Match-Logs'),
    ('Junya Ito','https://fbref.com/en/players/dbd3a428/matchlogs/2021-2022/c37/Junya-Ito-Match-Logs'),
    ('Luis Fernando Díaz Marulanda','https://fbref.com/en/players/4a1a9578/matchlogs/2021-2022/c32/Luis-Diaz-Match-Logs'),
    ('Hugo Cuypers','https://fbref.com/en/players/9f417f8c/matchlogs/2021-2022/Hugo-Cuypers-Match-Logs'),     
    ('Joshua Zirkzee','https://fbref.com/en/players/028e70b9/matchlogs/2021-2022/c37/Joshua-Zirkzee-Match-Logs'),
    ('Marlos Moreno Durán','https://fbref.com/en/players/8499c713/matchlogs/2021-2022/Marlos-Moreno-Match-Logs'), 
    ('Kieffer Moore','https://fbref.com/en/players/01ce9e70/matchlogs/2021-2022/c10/Kieffer-Moore-Match-Logs'),
    ('Charles De Ketelaere','https://fbref.com/en/players/2ef7c612/matchlogs/2021-2022/c37/Charles-De-Ketelaere-Match-Logs'),
    ('Georges Mikautadze','https://fbref.com/en/players/4788cb6b/matchlogs/2021-2022/c37/Georges-Mikautadze-Match-Logs'), 
    ('Tom Eaves','https://fbref.com/en/players/377c465b/matchlogs/2021-2022/c10/Tom-Eaves-Match-Logs'),
    ('Josh Bowler','https://fbref.com/en/players/ea39a081/matchlogs/2021-2022/c10/Josh-Bowler-Match-Logs'),
    ('Keane Lewis-Potter','https://fbref.com/en/players/41f08ac8/matchlogs/2021-2022/c10/Keane-Lewis-Potter-Match-Logs'),
    ('Harry Cornick','https://fbref.com/en/players/9169bc41/matchlogs/2021-2022/c10/Harry-Cornick-Match-Logs'),
    ('Antoine Semenyo','https://fbref.com/en/players/efd2ec23/matchlogs/2021-2022/c10/Antoine-Semenyo-Match-Logs'),
    ('Jacob Brown','https://fbref.com/en/players/ca18c8f1/matchlogs/2021-2022/c10/Jacob-Brown-Match-Logs'),
    ('Karim Adeyemi','https://fbref.com/en/players/501055d8/matchlogs/2021-2022/c56/Karim-Adeyemi-Match-Logs'),
    ('Abel Ruiz Ortega','https://fbref.com/en/players/6cbf8d0d/matchlogs/2021-2022/c32/Abel-Ruiz-Match-Logs'),
    ('Geoffry Hairemans','https://fbref.com/en/players/64aaee47/matchlogs/2021-2022/Geoffry-Hairemans-Match-Logs'),
    ('Dante Vanzeir','https://fbref.com/en/players/d682f554/matchlogs/2021-2022/c37/Dante-Vanzeir-Match-Logs'),
    ('Nikola Storm','https://fbref.com/en/players/6460189b/matchlogs/2021-2022/Nikola-Storm-Match-Logs'),
    ('Darwin Gabriel Núñez Ribeiro','https://fbref.com/en/players/4d77b365/matchlogs/2021-2022/c32/Darwin-Nunez-Match-Logs'),
    ('Rui Pedro  Silva Costa','https://fbref.com/en/players/ee43adcd/matchlogs/2021-2022/c32/Rui-Costa-Match-Logs'),
    ('Dario Tadić','https://fbref.com/en/players/1a933b10/matchlogs/2021-2022/Dario-Tadic-Match-Logs'),
    ('Lyle Taylor','https://fbref.com/en/players/3e80e04e/matchlogs/2021-2022/c10/Lyle-Taylor-Match-Logs'),
    ('Brennan Johnson','https://fbref.com/en/players/0cd31129/matchlogs/2021-2022/c10/Brennan-Johnson-Match-Logs'),
    ('Jason Knight','https://fbref.com/en/players/b83ec960/matchlogs/2021-2022/c10/Jason-Knight-Match-Logs'),
    ('Mallik Wilks','https://fbref.com/en/players/f2639dc1/matchlogs/2021-2022/Mallik-Wilks-Match-Logs'),
    ('Musashi Suzuki','https://fbref.com/en/players/53711e42/matchlogs/2021-2022/Musashi-Suzuki-Match-Logs'),
    ('Daichi Hayashi','https://fbref.com/en/players/b5611533/matchlogs/2021-2022/c37/Daichi-Hayashi-Match-Logs'),
    ('Marco Djuričin','https://fbref.com/en/players/a7f600c5/matchlogs/2021-2022/c56/Marco-Djuricin-Match-Logs'),
    ('Cassiano Dias Moreira','https://fbref.com/en/players/46fa8966/matchlogs/2021-2022/Cassiano-Match-Logs'),
    ('Gustavo Affonso Sauerbeck','https://fbref.com/en/players/fd53959e/matchlogs/2021-2022/c32/Gustavo-Sauer-Match-Logs'),
    ('Welinton Júnior Ferreira dos Santos','https://fbref.com/en/players/f1ef2def/matchlogs/2021-2022/c32/Welinton-Junior-Match-Logs'),
    ('Rúben Barcelos De Sousa Lameiras','https://fbref.com/en/players/656a96c6/matchlogs/2021-2022/Ruben-Lameiras-Match-Logs'),
    ('Walterson Silva','https://fbref.com/en/players/8eebb330/matchlogs/2021-2022/Walterson-Match-Logs'),
    ('Samuel Dias Lino','https://fbref.com/en/players/2a295d33/matchlogs/2021-2022/Samuel-Lino-Match-Logs'),
    ('Denílson Pereira Júnior','https://fbref.com/en/players/4447f575/matchlogs/2021-2022/c32/Denilson-Match-Logs'),
    ('Guilherme Schettine Guimarães','https://fbref.com/en/players/6b074beb/matchlogs/2021-2022/Guilherme-Schettine-Match-Logs'),
    ('Cryzan da Cruz Queiroz Barcelos','https://fbref.com/en/players/70acbadc/matchlogs/2021-2022/c32/Crysan-Match-Logs'),
    ('Emil Riis Jakobsen','https://fbref.com/en/players/292cdb26/matchlogs/2021-2022/c10/Emil-Riis-Jakobsen-Match-Logs'),
    ('Thibo Somers','https://fbref.com/en/players/64238658/matchlogs/2021-2022/Thibo-Somers-Match-Logs'),
    ('Dylan Mbayo','https://fbref.com/en/players/18d5f9e8/matchlogs/2021-2022/Dylan-Mbayo-Match-Logs'),
    ('Faïz Selemani','https://fbref.com/en/players/c3e587ec/matchlogs/2021-2022/c37/Faiz-Selemani-Match-Logs'),
    ('Pape Habib Guèye','https://fbref.com/en/players/8c5f4501/matchlogs/2021-2022/Pape-Habib-Gueye-Match-Logs'),
    ('Yuma Suzuki','https://fbref.com/en/players/9e98928a/matchlogs/2021-2022/c37/Yuma-Suzuki-Match-Logs'),
    ('Manfred Fischer','https://fbref.com/en/players/a681baa0/matchlogs/2021-2022/c56/Manfred-Fischer-Match-Logs'),
    ('Ercan Kara','https://fbref.com/en/players/540c9a17/matchlogs/2021-2022/c56/Ercan-Kara-Match-Logs'),
    ('Francisco Evanilson de Lima Barbosa','https://fbref.com/en/players/6f3cc2fe/matchlogs/2021-2022/c32/Evanilson-Match-Logs'),
    ('Aleksandar  Jukić','https://fbref.com/en/players/096326df/Aleksandar-Jukic'),
    ('Florian Rieder','https://fbref.com/en/players/84f78cc5/matchlogs/2021-2022/Florian-Rieder-Match-Logs'),
    ('Kelvin Yeboah','https://fbref.com/en/players/ea0893d6/matchlogs/2021-2022/c56/Kelvin-Yeboah-Match-Logs'),
    ('Sammie Szmodics','https://fbref.com/en/players/39944392/matchlogs/2021-2022/c10/Sammie-Szmodics-Match-Logs'),
    ('Lyndon Dykes','https://fbref.com/en/players/cd92357a/matchlogs/2021-2022/c10/Lyndon-Dykes-Match-Logs'),
    ('Alex Timossi Andersson','https://fbref.com/en/players/5e4dae5f/matchlogs/2021-2022/Alex-Timossi-Andersson-Match-Logs'),
    ('Julien Fontaine Ngoy Bin Cibambi','https://fbref.com/en/players/87449a74/matchlogs/2021-2022/Julien-Ngoy-Match-Logs'),
    ('Joël Mohammed Ramzan Piroe','https://fbref.com/en/players/1f0072a6/matchlogs/2021-2022/c10/Joel-Piroe-Match-Logs'),
    ('Gonçalo Matias Ramos','https://fbref.com/en/players/f63cda26/matchlogs/2021-2022/c32/Goncalo-Ramos-Match-Logs'),
    ('Festy Ebosele','https://fbref.com/en/players/dbfa4f16/matchlogs/2021-2022/c10/Festy-Ebosele-Match-Logs'),
    ('Reda Khadra','https://fbref.com/en/players/88acf83f/matchlogs/2021-2022/c10/Reda-Khadra-Match-Logs'),
    ('Arthur Gomes Lourenço','https://fbref.com/en/players/184ad240/matchlogs/2021-2022/c32/Arthur-Match-Logs'),
    ('Alisson Pelegrini Safira','https://fbref.com/en/players/bac8e258/matchlogs/2021-2022/c32/Safira-Match-Logs'),
    ('Eduardo Gabriel Aquino Cossa','https://fbref.com/en/players/fc4e7637/matchlogs/2021-2022/c32/Pepe-Match-Logs'),
    ('Lucas da Silva de Jesus','https://fbref.com/en/players/aa47c6ce/matchlogs/2021-2022/c32/Lucas-Silva-Match-Logs'),
    ('Iván Darío Angulo Cortés','https://fbref.com/en/players/55a5d959/matchlogs/2021-2022/Ivan-Angulo-Match-Logs'),
    ('Isaac Nuhu','https://fbref.com/en/players/617deb10/matchlogs/2021-2022/Isaac-Nuhu-Match-Logs'),
    ('Žan Rogelj','https://fbref.com/en/players/0f4f3267/matchlogs/2021-2022/c56/Zan-Rogelj-Match-Logs'),
    ('Stefan Nutz','https://fbref.com/en/players/ae3bd67d/matchlogs/2021-2022/Stefan-Nutz-Match-Logs'),
    ('Dario Vizinger','https://fbref.com/en/players/8d965034/matchlogs/2021-2022/Dario-Vizinger-Match-Logs'),
    ('Marco Grüll','https://fbref.com/en/players/8babb812/matchlogs/2021-2022/c56/Marco-Grull-Match-Logs'),
    ('Jackson  Muleka Kyanvubu','https://fbref.com/en/players/383e8f03/matchlogs/2021-2022/c37/Jackson-Muleka-Match-Logs'),
    ('Tarik Tissoudali','https://fbref.com/en/players/4f60a143/matchlogs/2021-2022/c37/Tarik-Tissoudali-Match-Logs'),
    ('Rasmus Winther Højlund','https://fbref.com/en/players/491a433d/matchlogs/2021-2022/c56/Rasmus-Hojlund-Match-Logs'),
    ('Matt Godden','https://fbref.com/en/players/4e43f8c0/matchlogs/2021-2022/c10/Matthew-Godden-Match-Logs'),
    ('Tyrhys Dolan','https://fbref.com/en/players/ce967905/matchlogs/2021-2022/c10/Tyrhys-Dolan-Match-Logs'),
    ('Jaidon Anthony','https://fbref.com/en/players/e4238fb1/matchlogs/2021-2022/c10/Jaidon-Anthony-Match-Logs'),
    ('Ali Alipour','https://fbref.com/en/players/488ee64c/matchlogs/2021-2022/c32/Ali-Alipour-Match-Logs'),
    ('Kanya Fujimoto','https://fbref.com/en/players/be040b84/matchlogs/2021-2022/Kanya-Fujimoto-Match-Logs'),
    ('Petar Musa','https://fbref.com/en/players/4567a0b4/matchlogs/2021-2022/c32/Petar-Musa-Match-Logs'),
    ('Francisco José Navarro Aliaga','https://fbref.com/en/players/02a5618e/matchlogs/2021-2022/Fran-Navarro-Match-Logs'),
    ('Mark Harris','https://fbref.com/en/players/fcc02804/matchlogs/2021-2022/c10/Mark-Harris-Match-Logs'),
    ('Mohamed Badamosi','https://fbref.com/en/players/2f90bb04/matchlogs/2021-2022/c37/Muhammed-Badamosi-Match-Logs'),
    ('Ante Bajić','https://fbref.com/en/players/f66491e4/matchlogs/2021-2022/Ante-Bajic-Match-Logs'),
    ('Vítor Manuel Carvalho Oliveira','https://fbref.com/en/players/74d4cec6/matchlogs/2021-2022/c32/Vitinha-Match-Logs'),
    ('Alejandro Millán Iranzo','https://fbref.com/en/players/29ad0dd4/matchlogs/2021-2022/Alex-Millan-Match-Logs'),
    ('Tai Baribo','https://fbref.com/en/players/18a8e594/matchlogs/2021-2022/c56/Thai-Baribo-Match-Logs'),
    ('Francisco Miguel Ribeiro Tomé Tavares Bondoso','https://fbref.com/en/players/eb60757f/matchlogs/2021-2022/Kiko-Bondoso-Match-Logs'),
    ('Junior Chukwubuike Adamu','https://fbref.com/en/players/82e12fe7/matchlogs/2021-2022/c56/Junior-Adamu-Match-Logs'),
    ('Ched Evans','https://fbref.com/en/players/bb770c52/matchlogs/2021-2022/c10/Ched-Evans-Match-Logs'),
    ('Philip Zinckernagel','https://fbref.com/en/players/a5ea254c/matchlogs/2021-2022/c10/Philip-Zinckernagel-Match-Logs'),
    ('Noah Ohio','https://fbref.com/en/players/47ad225e/matchlogs/2021-2022/Noah-Ohio-Match-Logs'),
    ('Allano Brendon de Souza Lima','https://fbref.com/en/players/9c1245f8/matchlogs/2021-2022/c32/Allano-Match-Logs'),
    ('Sorba Thomas','https://fbref.com/en/players/8725a42e/matchlogs/2021-2022/c10/Sorba-Thomas-Match-Logs'),
    ('Iliman Cheikh Ndiaye','https://fbref.com/en/players/5ed97752/matchlogs/2021-2022/c10/Iliman-Ndiaye-Match-Logs'),
    ('Elijah Adebayo','https://fbref.com/en/players/0480ebc7/matchlogs/2021-2022/c10/Elijah-Adebayo-Match-Logs'),
    ('Taichi Hara','https://fbref.com/en/players/1e524738/matchlogs/2021-2022/c37/Taichi-Hara-Match-Logs'),
    ('Isaak Davies','https://fbref.com/en/players/8199addf/matchlogs/2021-2022/c10/Isaak-Davies-Match-Logs'),
    ('Darko Lemajić','https://fbref.com/en/players/6d207bd8/matchlogs/2021-2022/c37/Darko-Lemajic-Match-Logs'),
    ('Aron Dønnum','https://fbref.com/en/players/10cff752/matchlogs/2021-2022/c37/Aron-Donnum-Match-Logs'),
    ('Deniz Undav','https://fbref.com/en/players/dd549382/matchlogs/2021-2022/Deniz-Undav-Match-Logs'),
    ('Anass Zaroury','https://fbref.com/en/players/07d65234/matchlogs/2021-2022/Anass-Zaroury-Match-Logs'),
    ('Suliman Marlon Mustapha','https://fbref.com/en/players/9ff8ec4a/matchlogs/2021-2022/Marlon-Mustapha-Match-Logs'),
    ('Nuno Gonçalo Rocha Moreira','https://fbref.com/en/players/c28df63c/matchlogs/2021-2022/Nuno-Goncalo-Moreira-Match-Logs'),
    ('Arsénio Martins Lafuerte Nunes','https://fbref.com/en/players/50e148af/matchlogs/2021-2022/Arsenio-Martins-Lafuente-Nunes-Match-Logs'),
    ('André Oliveira e Silva','https://fbref.com/en/players/dde93e2c/matchlogs/2021-2022/Andre-Silva-Match-Logs'),
    ('André Clóvis Silva Filho','https://fbref.com/en/players/42995526/matchlogs/2021-2022/Andre-Clovis-Match-Logs'),
    ('Jonson Clarke-Harris','https://fbref.com/en/players/260af28b/matchlogs/2021-2022/c10/Jonson-Clarke-Harris-Match-Logs'),
    ('CJ Hamilton','https://fbref.com/en/players/63ffd16e/matchlogs/2021-2022/c10/CJ-Hamilton-Match-Logs'),
    ('Keshi Anderson','https://fbref.com/en/players/864a3eba/matchlogs/2021-2022/c10/Keshi-Anderson-Match-Logs'),
    ('Jerry Yates','https://fbref.com/en/players/637047f7/matchlogs/2021-2022/c10/Jerry-Yates-Match-Logs'), 
    ('Daniel da Silva dos Anjos','https://fbref.com/en/players/fc2a8364/matchlogs/2021-2022/Daniel-Match-Logs'),
    ('Ricardo Jorge Oliveira António','https://fbref.com/en/players/e472d4da/matchlogs/2021-2022/c32/Ricardinho-Match-Logs'),
    ('Siriki Dembélé','https://fbref.com/en/players/11f54bd5/matchlogs/2021-2022/c10/Siriki-Dembele-Match-Logs'),
    ('Cameron Archer','https://fbref.com/en/players/05e8ca6d/matchlogs/2021-2022/c10/Cameron-Archer-Match-Logs'),
    ('Abel Issa Camará','https://fbref.com/en/players/be80b225/matchlogs/2021-2022/Abel-Camara-Match-Logs'),
    ('Luke  Plange','https://fbref.com/en/players/2140e999/matchlogs/2021-2022/c10/Luke-Plange-Match-Logs'),
    ('Billal Messaoudi','https://fbref.com/en/players/469ddf42/matchlogs/2021-2022/Billel-Messaoudi-Match-Logs'), 
    ('Malcolm Ebiowei','https://fbref.com/en/players/fa9fa773/matchlogs/2021-2022/c10/Malcolm-Ebiowei-Match-Logs'), 
    ('Bernhard Zimmermann','https://fbref.com/en/players/41fffd24/matchlogs/2021-2022/c56/Bernhard-Zimmermann-Match-Logs'),
    ('Raphael Holzhauser','https://fbref.com/en/players/2f9c21da/matchlogs/2021-2022/Raphael-Holzhauser-Match-Logs'),
    ('Andreas Weimann','https://fbref.com/en/players/5996ffd9/matchlogs/2021-2022/c10/Andreas-Weimann-Match-Logs'),

    ]

all_players_data = []

for player_name, player_url in player_list:
    try:
        print(f"Processing data for {player_name}")
        player_summary = process_player_stats(player_url, player_name)
        all_players_data.append(player_summary)
        time.sleep(5)  # Add a delay between requests
    except HTTPError as e:
        print(f"Error fetching data for {player_name}: {e}")
        continue  # Skip to the next player

# Print all player data to check progress
print("All Players Data:")
print(all_players_data)


Processing data for Aleksandar Mitrović
Processing data for André Watshini Bukia
Processing data for Bas Dost
Processing data for Benik Afobe
Processing data for Benito Raman
Processing data for Carlton Morris
Processing data for Cauley Woodrow
Processing data for Chris Martin
Processing data for Dominic Solanke
Processing data for Duncan Watmore
Processing data for Jakob Jantscher
Processing data for James Collins
Processing data for Jean-Luc Mamadou Diarra Dompé
Processing data for Jelle Vossen
Processing data for Jhon Eduard Murillo Romaña
Processing data for Josh Magennis
Processing data for Kenji Gorré
Processing data for Laurent Depoitre
Processing data for Lawrence Shankland
Processing data for Lewis Grabban
Processing data for Lior Refaelov
Processing data for Lucas Eduardo Santos João
Processing data for Lukas Jutkiewicz
Processing data for Manuel Hedilazio Benson
Processing data for Michael Frey
Processing data for Neeskens Kebano
Processing data for Pablo Sarabia García
Proc

In [2]:
all_players_data

[                Player  Team_points   Gls  Ast     Sh   SoT
 0  Aleksandar Mitrović         86.0  43.0  7.0  191.0  83.0,
                  Player  Team_points  Gls  Ast  Sh  SoT
 0  André Watshini Bukia           27    4    2  54   11,
      Player  Team_points  Gls  Ast  Sh  SoT
 0  Bas Dost           16   11    0  26   15,
         Player  Team_points  Gls  Ast  Sh  SoT
 0  Benik Afobe           47   12    3  66   32,
          Player  Team_points  Gls  Ast    Sh  SoT
 0  Benito Raman         18.0  3.0  2.0  19.0  4.0,
            Player  Team_points  Gls  Ast    Sh   SoT
 0  Carlton Morris         20.0  7.0  3.0  60.0  25.0,
            Player  Team_points  Gls  Ast    Sh   SoT
 0  Cauley Woodrow         13.0  4.0  1.0  68.0  28.0,
          Player  Team_points  Gls  Ast  Sh  SoT
 0  Chris Martin           52   12    6  80   32,
             Player  Team_points  Gls  Ast   Sh  SoT
 0  Dominic Solanke           88   29    7  159   54,
            Player  Team_points  Gls  Ast  Sh  

In [3]:
# reshape all_players_data into a 2D array
import numpy as np
all_players_data = np.array(all_players_data).reshape(239, 6)

In [4]:
# convert to  dataframe
players_data = pd.DataFrame(all_players_data)
players_data

,0,1,2,3,4,5
0,Aleksandar Mitrović,86.0,43.0,7.0,191.0,83.0
1,André Watshini Bukia,27,4,2,54,11
2,Bas Dost,16,11,0,26,15
3,Benik Afobe,47,12,3,66,32
4,Benito Raman,18.0,3.0,2.0,19.0,4.0
...,...,...,...,...,...,...
234,Billal Messaoudi,6.0,1.0,1.0,37.0,11.0
235,Malcolm Ebiowei,13,1,2,22,6
236,Bernhard Zimmermann,17,4,0,25,9
237,Raphael Holzhauser,10,3,4,35,9


In [5]:
# change column names
players_data.columns = ['Player', 'points_won', 'Goals', 'Assists', 'Shots', 'Shots_on_target']
players_data

,Player,points_won,Goals,Assists,Shots,Shots_on_target
0,Aleksandar Mitrović,86.0,43.0,7.0,191.0,83.0
1,André Watshini Bukia,27,4,2,54,11
2,Bas Dost,16,11,0,26,15
3,Benik Afobe,47,12,3,66,32
4,Benito Raman,18.0,3.0,2.0,19.0,4.0
...,...,...,...,...,...,...
234,Billal Messaoudi,6.0,1.0,1.0,37.0,11.0
235,Malcolm Ebiowei,13,1,2,22,6
236,Bernhard Zimmermann,17,4,0,25,9
237,Raphael Holzhauser,10,3,4,35,9


In [6]:
# convert columns to numeric
columns_to_convert = ['points_won', 'Goals', 'Assists', 'Shots', 'Shots_on_target']
players_data[columns_to_convert] = players_data[columns_to_convert].apply(pd.to_numeric, errors='coerce')

In [7]:
# check data types
players_data.dtypes

Player              object
points_won         float64
Goals              float64
Assists            float64
Shots              float64
Shots_on_target    float64
dtype: object

In [8]:
players_data

,Player,points_won,Goals,Assists,Shots,Shots_on_target
0,Aleksandar Mitrović,86.0,43.0,7.0,191.0,83.0
1,André Watshini Bukia,27.0,4.0,2.0,54.0,11.0
2,Bas Dost,16.0,11.0,0.0,26.0,15.0
3,Benik Afobe,47.0,12.0,3.0,66.0,32.0
4,Benito Raman,18.0,3.0,2.0,19.0,4.0
...,...,...,...,...,...,...
234,Billal Messaoudi,6.0,1.0,1.0,37.0,11.0
235,Malcolm Ebiowei,13.0,1.0,2.0,22.0,6.0
236,Bernhard Zimmermann,17.0,4.0,0.0,25.0,9.0
237,Raphael Holzhauser,10.0,3.0,4.0,35.0,9.0


In [9]:
# check null
players_data.isnull().sum().sum()

0

In [10]:
# save to csv
players_data.to_csv('players_data.csv', index=False)

# Get league tables

In [22]:
import pandas as pd

# fbref table link
Championship_table = 'https://fbref.com/en/comps/10/2021-2022/2021-2022-Championship-Stats'

# convert html table to pandas dataframe
Championship_table = pd.read_html(Championship_table)

# Convert list of tables to DataFrame
Championship_table = pd.DataFrame(Championship_table[0])

# Selecting specific columns
Championship_table = Championship_table[['Squad', 'GF', 'Pts', 'Top Team Scorer']]

# Splitting 'Top Team Scorer' column and selecting the first part
Championship_table['Top Team Scorer'] = Championship_table['Top Team Scorer'].str.split('-', expand=True)[0]

# Renaming column
Championship_table.rename(columns={'Squad': 'Team', 'GF': 'Goals', 'Pts': 'Points', 'Top Team Scorer': 'Top_scorer'}, inplace=True)

# Display the DataFrame
Championship_table

,Team,Goals,Points,Top_scorer
0,Fulham,106,90,Aleksandar Mitrović
1,Bournemouth,74,88,Dominic Solanke
2,Huddersfield,64,82,Danny Ward
3,Nott'ham Forest,73,80,Brennan Johnson
4,Sheffield Utd,63,75,Billy Sharp
5,Luton Town,63,75,Elijah Adebayo
6,Middlesbrough,59,70,Matt Crooks
7,Blackburn,59,69,Ben Brereton
8,Millwall,53,69,Benik Afobe
9,West Brom,52,67,Karlan Grant


In [11]:
import pandas as pd

# fbref table link
Primeira_Liga_table = 'https://fbref.com/en/comps/32/2021-2022/2021-2022-Primeira-Liga-Stats'

# convert html table to pandas dataframe
Primeira_Liga_table = pd.read_html(Primeira_Liga_table)

# Convert list of tables to DataFrame
Primeira_Liga_table = pd.DataFrame(Primeira_Liga_table[0])

# Selecting specific columns
Primeira_Liga_table = Primeira_Liga_table[['Squad', 'GF', 'Pts', 'Top Team Scorer']]

# Splitting 'Top Team Scorer' column and selecting the first part
Primeira_Liga_table['Top Team Scorer'] = Primeira_Liga_table['Top Team Scorer'].str.split('-', expand=True)[0]

# Renaming column
Primeira_Liga_table.rename(columns={'Squad': 'Team', 'GF': 'Goals', 'Pts': 'Points', 'Top Team Scorer': 'Top_scorer'}, inplace=True)

# Display the DataFrame
Primeira_Liga_table

,Team,Goals,Points,Top_scorer
0,Porto,86,91,Mehdi Taremi
1,Sporting CP,73,85,Pablo Sarabia
2,Benfica,78,74,Darwin Núñez
3,Braga,52,65,Ricardo Horta
4,Gil Vicente FC,47,51,Fran Navarro
5,Vitória,50,48,Oscar Estupiñan
6,Santa Clara,38,40,Crysan
7,Famalicão,45,39,Simon Banza
8,Estoril,36,39,André Franco
9,Marítimo,39,38,Diederrick Joel


In [12]:
import pandas as pd

# fbref table link
Belgian_pro_table = 'https://fbref.com/en/comps/37/2021-2022/2021-2022-Belgian-Pro-League-Stats'

# convert html table to pandas dataframe
Belgian_pro_table = pd.read_html(Belgian_pro_table)

# Convert list of tables to DataFrame
Belgian_pro_table = pd.DataFrame(Belgian_pro_table[0])

# Selecting specific columns
Belgian_pro_table = Belgian_pro_table[['Squad', 'GF', 'Pts', 'Top Team Scorer']]

# Splitting 'Top Team Scorer' column and selecting the first part
Belgian_pro_table['Top Team Scorer'] = Belgian_pro_table['Top Team Scorer'].str.split('-', expand=True)[0]

# Renaming column
Belgian_pro_table.rename(columns={'Squad': 'Team', 'GF': 'Goals', 'Pts': 'Points', 'Top Team Scorer': 'Top_scorer'}, inplace=True)

# Display the DataFrame
Belgian_pro_table

,Team,Goals,Points,Top_scorer
0,Union SG,78.0,77.0,Deniz Undav
1,Club Brugge,72.0,72.0,Charles De Ketelaere
2,Anderlecht,72.0,64.0,Joshua Zirkzee
3,Antwerp,55.0,63.0,Michael Frey
4,NaN,NaN,NaN,NaN
5,Gent,56.0,62.0,Tarik Tissoudali
6,Charleroi,55.0,54.0,Shamar Nicholson
7,Mechelen,57.0,52.0,Nikola Storm
8,Genk,66.0,51.0,Paul Onuachu
9,NaN,NaN,NaN,NaN


In [19]:
import pandas as pd

# fbref table link
Aust_Bundesliga_table = 'https://fbref.com/en/comps/56/2021-2022/2021-2022-Austrian-Bundesliga-Stats'

# convert html table to pandas dataframe
Aust_Bundesliga_table = pd.read_html(Aust_Bundesliga_table)

# Convert list of tables to DataFrame
Aust_Bundesliga_table = pd.DataFrame(Aust_Bundesliga_table[0])

# Selecting specific columns
Aust_Bundesliga_table = Aust_Bundesliga_table[['Squad', 'GF', 'Pts', 'Top Team Scorer']]

# Splitting 'Top Team Scorer' column and selecting the first part
Aust_Bundesliga_table['Top Team Scorer'] = Aust_Bundesliga_table['Top Team Scorer'].str.split('-', expand=True)[0]

# Renaming column
Aust_Bundesliga_table.rename(columns={'Squad': 'Team', 'GF': 'Goals', 'Pts': 'Points', 'Top Team Scorer': 'Top_scorer'}, inplace=True)

# Display the DataFrame
Aust_Bundesliga_table

,Team,Goals,Points,Top_scorer
0,RB Salzburg,77.0,80.0,Karim Adeyemi
1,Sturm Graz,62.0,56.0,Jakob Jantscher
2,Austria Wien,44.0,46.0,Marco Djuricin
3,Wolfsberger AC,48.0,47.0,Thai Baribo
4,Rapid Wien,48.0,41.0,Ercan Kara
5,SK Austria Klagenfurt,43.0,36.0,Markus Pink
6,NaN,NaN,NaN,NaN
7,WSG Tirol,46.0,40.0,Giacomo Vrioni
8,LASK,44.0,39.0,"Thomas Goiginger, Sascha Horvath"
9,SCR Altach,24.0,29.0,Atdhe Nuhiu


In [20]:
Aust_Bundesliga_table = Aust_Bundesliga_table.dropna()

# Remove "Sascha Horvath" from Top_scorer where Team is LASK
Aust_Bundesliga_table.loc[Aust_Bundesliga_table['Team'] == 'LASK', 'Top_scorer'] = Aust_Bundesliga_table.loc[Aust_Bundesliga_table['Team'] == 'LASK', 'Top_scorer'].str.split(',').str[0]

# Reset index
Aust_Bundesliga_table.reset_index(drop=True, inplace=True)

Aust_Bundesliga_table


,Team,Goals,Points,Top_scorer
0,RB Salzburg,77.0,80.0,Karim Adeyemi
1,Sturm Graz,62.0,56.0,Jakob Jantscher
2,Austria Wien,44.0,46.0,Marco Djuricin
3,Wolfsberger AC,48.0,47.0,Thai Baribo
4,Rapid Wien,48.0,41.0,Ercan Kara
5,SK Austria Klagenfurt,43.0,36.0,Markus Pink
6,WSG Tirol,46.0,40.0,Giacomo Vrioni
7,LASK,44.0,39.0,Thomas Goiginger
8,SCR Altach,24.0,29.0,Atdhe Nuhiu
9,Ried,40.0,37.0,Ante Bajic


In [23]:
# convert to csv
Championship_table.to_csv('Championship_table.csv', index=False)
Primeira_Liga_table.to_csv('Primeira_Liga_table.csv', index=False)
Belgian_pro_table.to_csv('Belgian_pro_table.csv', index=False)
Aust_Bundesliga_table.to_csv('Aust_Bundesliga_table.csv', index=False)